In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import model_from_json
from flask import Flask, redirect, url_for, request
import numpy as np
import pandas as pd

In [2]:
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    my_df = pd.read_csv("Policies.csv")
    age=22
    sex=0
    bmi=22.8
    noc=0
    smoker=0
    Xnew=np.array([[age,sex,bmi,noc,smoker]])
    Ynew=loaded_model.predict(Xnew)
    price=int(Ynew[0])
    rupees=price/2*71.4
    print(my_df.loc[(my_df['Min Entry age'] < age) & (my_df['Max Entry Age'] > age) & (my_df['Max Sum Assured'] > rupees) & (my_df['Min Sum Assured'] < rupees)])

Loaded model from disk
                     Policy Name  Min Entry age  Max Entry Age  \
2   Aditya Birla Health Inurance           0.24            100   
6      Max Bupa Health Insurance           5.00             50   
7  IFFCO Tokio Health Insurance           12.00             70   
8          SBI Health Insurance           18.00             65   

   Min Sum Assured  Max Sum Assured  
2            50000          1000000  
6            50000          5000000  
7            10000           100000  
8            50000           500000  


In [ ]:
app=Flask(__name__)
@app.route('/estimate/')
def predictpolicy():
    age=request.args.get('age')
    sex=request.args.get('sex')
    bmi=request.args.get('bmi')
    noc=request.args.get('noc')
    smoker=request.args.get('smoker')
    my_df = pd.read_csv("Policies.csv")
    Xnew=np.array([[age,sex,bmi,noc,smoker]])
    Ynew=loaded_model.predict(Xnew)
    price=int(Ynew[0])
    rupees=price/2*71.4
    print(my_df.loc[(my_df['Min Entry age'] < int(age)) & (my_df['Max Entry Age'] > int(age)) & (my_df['Max Sum Assured'] > float(rupees)) & (my_df['Min Sum Assured'] < float(rupees))])
    return 'Estimated value=%f ' % rupees
@app.route('/policy',methods=['POST','GET']) 
def policy():
    if request.method=='POST':
        a=request.form['age']
        s=request.form['sex']
        b=request.form['bmi']
        n=request.form['noc']
        sm=request.form['sm']
        return redirect(url_for('predictpolicy',age=a,sex=s,bmi=b,noc=n,smoker=sm))
    else:
        a=request.args.get('age')
        s=request.args.get('sex')
        b=request.args.get('bmi')
        n=request.args.get('noc')
        sm=request.args.get('sm')
        return redirect(url_for('predictpolicy',age=a,sex=s,bmi=b,noc=n,smoker=sm))
if __name__=='__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2018 02:59:30] "POST /policy HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2018 02:59:30] "GET /estimate/?age=22&sex=1&bmi=22.8&noc=3&smoker=0 HTTP/1.1" 200 -


                    Policy Name  Min Entry age  Max Entry Age  \
0       Apollo Munich Easy plan           5.00             65   
2  Aditya Birla Health Inurance           0.24            100   
6     Max Bupa Health Insurance           5.00             50   
8         SBI Health Insurance           18.00             65   

   Min Sum Assured  Max Sum Assured  
0           100000          5000000  
2            50000          1000000  
6            50000          5000000  
8            50000           500000  


127.0.0.1 - - [21/Nov/2018 03:02:33] "POST /policy HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2018 03:02:33] "GET /estimate/?age=22&sex=1&bmi=22.8&noc=3&smoker=0 HTTP/1.1" 200 -


                    Policy Name  Min Entry age  Max Entry Age  \
0       Apollo Munich Easy plan           5.00             65   
2  Aditya Birla Health Inurance           0.24            100   
6     Max Bupa Health Insurance           5.00             50   
8         SBI Health Insurance           18.00             65   

   Min Sum Assured  Max Sum Assured  
0           100000          5000000  
2            50000          1000000  
6            50000          5000000  
8            50000           500000  
